## Swin Transformer

- Functions:
  1. window partition
  2. window reverse
- Classes:
  1. Window Attention
  2. Swin Block
  3. Patch Marge
  4. Multi-stage Swin Transformer
- Datasets
- Train + Test

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

## Functions:

- window partition
- window reverse


In [ ]:
def window_partition(x, win):
  """
  window partition function for Swin Transformer

  input:
  - x: (B, H, W, C)
  - win: window size

  output:
  - x: (B * num_wins, win, win, C)

  """
  B, H, W, C = x.shape
  x = x.view(B, H//win, win, W//win, win, C)
  x = x.permute(0, 1, 3, 2, 4, 5).contiguous()
  x = x.view(-1, win, win, C)
  # (win, win) == window size, (H//win, W//win) == total number of windows
  return x

def window_reverse(windows, win, W, H):
  """
  window reverse function for Swin Transformer

  input:
  - windows: (B, H, W, C)
  - win: window size
  - W: width of image
  - H: height of image

  output:
  - x: (B, H, W, C)
  """
  B = int(windows.shape[0] / (H//win * W//win))
  x = windows.view(B, H//win, W//win, win, win, -1)
  x.permute(0, 1, 3, 2, 4, 5).contiguous()
  x = x.view(B, H, W, -1)
  return x

In [ ]:
class WindowAttention(nn.Module):
  """
  Window Attention Module for Swin Transformer

  input:
  - x: (B, H, W, C)
  - win: window size

  """

  def __init__(self, dim, num_heads, win):
    super().__init__()
    self.dim = dim
    self.num_heads = num_heads
    self.head_dim = self.dim // self.num_heads
    self.scale = self.head_dim ** -0.5
    self.win = win

    self.q = nn.Linear(self.dim, self.dim)
    self.k = nn.Linear(self.dim, self.dim)
    self.v = nn.Linear(self.dim, self.dim)

    self.proj = nn.Linear(self.dim, self.dim)

    # relative positional bias
    coords = torch.stack(
        torch.meshgrid(
            torch.arange(self.win),
            torch.arange(self.win),
            indexing='ij'
        )
    )
    coords_flat = coords.flatten(1)

    rel = coords_flat[:, :, None] - coords_flat[:, None, :]

    rel = rel.permute(1, 2, 0)
    rel[:, :, 0] += (self.win - 1)
    rel[:, :, 1] += (self.win - 1)

    rel[:, :, 0] *= (2 * self.win - 1)
    index = rel.sum(-1)

    self.register_buffer('pos_index', index)
    self.rel_bias = nn.Parameter(torch.zeros(2 * self.win - 1, 2 * self.win - 1, self.num_heads))
    # mask value


  def forward(self, x, mask=None):

    B_, N, C = x.shape
    q = self.q(x)
    k = self.k(x)
    v = self.v(x)

    q = q * self.scale
    attn = q @ k.transpose(-2, -1)

    rb = self.rel_bias[self.position_index.view(-1)].view(N, N, -1)
    attn = attn + rb.permute(2, 0, 1).unsqueeze(0)

    if mask is not None:
      nw = mask.shape[0]
      attn = attn.view(B_//nw, nw, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
      attn = attn.view(-1, self.num_heads, N, N)

    attn = F.softmax(attn, dim=-1)
    out = attn @ v
    out = out.transpose(1, 2).reshape(B_, N, C)
    out = self.proj(out)
    return out



## examples of rel

In [ ]:
win=3
coords = torch.stack(
        torch.meshgrid(
            torch.arange(3),
            torch.arange(3),
            indexing='ij'
        )
    )

print("coords: ", coords)
coords_flat = coords.flatten(1)
print("\n")
print("coords_flat: ", coords_flat)
print("\n")
rel = coords_flat[:, :, None] - coords_flat[:, None, :]
print("rel: ", rel)
rel = rel.permute(1, 2, 0)
rel[:, :, 0] += (win - 1)
rel[:, :, 1] += (win - 1)
rel[:, :, 0] *= (2 * win - 1)
print(rel)
index = rel.sum(-1)
print(index)

coords:  tensor([[[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2]],

        [[0, 1, 2],
         [0, 1, 2],
         [0, 1, 2]]])


coords_flat:  tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2],
        [0, 1, 2, 0, 1, 2, 0, 1, 2]])


rel:  tensor([[[ 0,  0,  0, -1, -1, -1, -2, -2, -2],
         [ 0,  0,  0, -1, -1, -1, -2, -2, -2],
         [ 0,  0,  0, -1, -1, -1, -2, -2, -2],
         [ 1,  1,  1,  0,  0,  0, -1, -1, -1],
         [ 1,  1,  1,  0,  0,  0, -1, -1, -1],
         [ 1,  1,  1,  0,  0,  0, -1, -1, -1],
         [ 2,  2,  2,  1,  1,  1,  0,  0,  0],
         [ 2,  2,  2,  1,  1,  1,  0,  0,  0],
         [ 2,  2,  2,  1,  1,  1,  0,  0,  0]],

        [[ 0, -1, -2,  0, -1, -2,  0, -1, -2],
         [ 1,  0, -1,  1,  0, -1,  1,  0, -1],
         [ 2,  1,  0,  2,  1,  0,  2,  1,  0],
         [ 0, -1, -2,  0, -1, -2,  0, -1, -2],
         [ 1,  0, -1,  1,  0, -1,  1,  0, -1],
         [ 2,  1,  0,  2,  1,  0,  2,  1,  0],
         [ 0, -1, -2,  0, -1, -2,  0, -1, -2],
         [ 1, 

In [15]:
class SwinBlock(nn.Module):
  def __init__(self, dim, res, win, shift, heads):

    self.dim = dim
    self.res = res
    self.win = win
    self.shift = shift

    self.norm1 = nn.LayerNorm(dim)
    self.attn = WindowAttention(dim, res, win, heads)
    self.norm2 = nn.LayerNorm(dim)
    self.mlp = nn.Sequential(
        nn.Linear(dim, dim * 4),
        nn.GELU(),
        nn.Linear(dim * 4, dim)
    )

    H, W = res
    if shift > 0:
      self.mask = self.create_mash(H, W, win, shift)

    else:
      self.mask = None

    def create_mask(self, H, W, win, shift):

      mask = torch.zeros((1, H, W, 1))
      count = 0
      for h in (slice(0, -win), slice(- win, -shift), slice(-shift, None)):
        for w in (slice(0, -win), slice(- win, -shift), slice(-shift, None)):
          mask[:, h, w, :] = count
          count += 1

      mask = window_partition(mask, win)
      mask = mask.view(-1, win * win)
      mask = mask.unsqueeze(1) - mask.unsqueeze(2)
      mask = mask.masked_fill(mask!=0, -10000.0)
      return mask

  def forward(self, x):
    B, L, C = x.shape
    H, W = self.res

    shortcut = x
    x = self.norm1(x)
    x = x.view(B, H, W, C)

    if self.shift > 0:
      x = torch.roll(x, shifts=(-self.shift, -self.shift), dims=(1, 2))
      x[:, -self.shift:, -self.shift:, :] = -1

    win_x = window_partition(x, self.win).view(-1, self.win * self.win, C)

    mask = self.mask.to(x.device) if self.mask is not None else None
    win_x = self.attn(win_x, mask=mask)

    x = window_reverse(win_x, self.win, H, W)

    if self.shift > 0:
      x = torch.roll(x, shifts=(self.shift, self.shift), dims=(1, 2))

    x = x[:, :H, :W, :] + shortcut
    shortcut = x

    x = self.norm2(x)

    x = self.mlp(x)

    x = x + shortcut

    return x


In [ ]:
class PatchMerge(nn.Module):
  def __init__(self, dim):
    self.dim = dim

    self.reduction = nn.Linear(dim * 4, dim*2, bias=False)
    self.norm = nn.LayerNorm(dim)

  def forward(self, x, H, W):
    B, L, C = x.shape
    x = x.view(B, H, W, C)
    x0 = x[:, 0::2, 0::2, :]
    x1 = x[:, 1::2, 0::2, :]
    x2 = x[:, 0::2, 1::2, :]
    x3 = x[:, 1::2, 1::2, :]

    x0 = x0.view(B, -1, C)
    x1 = x1.view(B, -1, C)
    x2 = x2.view(B, -1, C)
    x3 = x3.view(B, -1, C)

    x = torch.cat([x0, x1, x2, x3], dim=-1)
    x = x.view(B, -1, C * 4)
    x = self.reduction(x)
    x = self.norm(x)
    return x, H//2, W//2

In [16]:
import torch
import torch.nn as nn

class TwoStageSwinMNIST(nn.Module):
    def __init__(self, embed_dim=48, heads=3, win=7, num_classes=10):
        super().__init__()

        self.embed_dim = embed_dim
        self.num_classes = num_classes

        # Patch embedding: 28x28 -> 14x14 with C = embed_dim
        self.patch_embed = nn.Conv2d(1, embed_dim, kernel_size=2, stride=2)
        initial_res = (14, 14)  # just for reference / debugging

        # ----- stage 1 -----
        # operates at 14x14 with dim = embed_dim
        self.stage1_blocks = nn.Sequential(
            SwinBlock(embed_dim, heads=heads, win=win, shift=0),
            SwinBlock(embed_dim, heads=heads, win=win, shift=3),
        )

        # Patch merging: 14x14 -> 7x7, dim: embed_dim -> 2 * embed_dim
        self.patch_merge1 = PatchMerging(embed_dim)

        # ----- stage 2 -----
        # operates at 7x7 with dim = 2 * embed_dim
        dim2 = 2 * embed_dim
        self.stage2_blocks = nn.Sequential(
            SwinBlock(dim2, heads=heads, win=win, shift=0),
            SwinBlock(dim2, heads=heads, win=win, shift=3),
        )

        # Classification head
        self.norm = nn.LayerNorm(dim2)
        self.head = nn.Linear(dim2, num_classes)

    def forward(self, x):
        """
        x: (B, 1, 28, 28)
        """
        # --- patch embedding ---
        x = self.patch_embed(x)         # (B, C, 14, 14), C = embed_dim
        B, C, H, W = x.shape

        # flatten to sequence: (B, H*W, C)
        x = x.flatten(2).transpose(1, 2)  # (B, 196, C)

        # --- stage 1 ---
        x = self.stage1_blocks(x)        # still (B, 196, C)

        # --- patch merge 1: 14x14 -> 7x7, dim x2 ---
        x = self.patch_merge1(x)         # (B, 49, 2C)

        # --- stage 2 ---
        x = self.stage2_blocks(x)        # (B, 49, 2C)

        # --- global pooling + classifier ---
        x = self.norm(x)                 # (B, 49, 2C)
        x = x.mean(dim=1)                # (B, 2C)  global average over tokens
        logits = self.head(x)            # (B, num_classes)

        return logits
